<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qworld.net" target="_blank"><img src="../images/qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by <a href="https://iitis.pl/pl/person/aglos" target="_blank"  >Adam Glos</a> and Özlem Salehi
        </td>        
</tr></table>

<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $

#  Grover Algorithm for Max-Cut Problem

Finally, we are ready to implement the Grover algorithm for the Max-Cut problem. 

For the initialization, we will apply Hadamard gate to the qubits representing the color of each vertex. We know how to implement the Grover diffusion operator. Now we need to implement an oracle which will "mark" the colorings in which there are $k$ edges whose endpoints are colored using different colors.

For the oracle, the procedure goes as follows:

* Implement edge checking for each edge (check whether an edge has endpoints with different colors).
* Sum the outputs of edge checking step.
* Check whether the sum is equal to (or greater) $k$ and save the flag on the auxillary qubit.
* Apply $Z$ gate on the auxillary qubit.

The oracle works for the decision version of the problem which checks whether there exists a coloring such that there are at least $k$ edges whose endpoints are colored using different colors. How to find the maximum size for the cut?

Suppose that $k$ is a three-digit number $b_2b_1b_0$. First, we should check whether there is a coloring such that $b_2$ is equal to 1. If it is, then we check if there is a coloring such that $b_2=1$ and $b_1=1$. Otherwise, we should check for the coloring such that $b_2=0$ and $b_1$=1. This way, by doing the binary search over binary representation, we can finally find the cut of maximal size.

Let us consider the following graph.
<img src="../images/path3.png" width="25%" align="center">

We will check whether there is a coloring such that two edges connect vertices with a different color. To check that we will verify, whether for number $b_1b_0$, $b_1$ is set to 1. We will use:
* Three qubits (0-2) for vertices.
* Two qubits (3-4) for edges.
* Two qubits (5-6) for storing the sum.
* Single qubit (7) for the auxillary qubit.

Since qubit 7 stores the flag, that is whether $b_1$ is set to 1 or not, we will apply the $Z$ gate on that qubit.

Below we present the final Grover algorithm. Note that there are two such solutions, so we apply the oracle only once.

In [1]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

def oracle_computation():
    #edge checking
    yield CX(qq[0], qq[3])
    yield CX(qq[1], qq[3])
    yield CX(qq[1], qq[4])
    yield CX(qq[2], qq[4])
    
    #adding qubit 3
    yield CX(qq[3], qq[5])
    
    #adding qubit 4
    yield CCX(qq[4], qq[5], qq[6])
    yield CX(qq[4], qq[5])
    
    #check if b1 is equal to 1 and store the result in the auxillary qubit
    yield CX(qq[6], qq[7])

def oracle():
    yield oracle_computation()
    yield Z(qq[7])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:3]))
    yield X.on_each(*(qq[:3]))
    yield Z(qq[2]).controlled_by(qq[0], qq[1])
    yield X.on_each(*(qq[:3]))
    yield H.on_each(*(qq[:3]))


# the Grover algorithm
qq = cirq.LineQubit.range(8)
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:3])))

for i in range(1):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
    
circuit.append(cirq.measure(*qq[0:3], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)

print("Random guess probability:", 1./2**3)

# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of obsering", state, ": " ,c/trials_number)

Random guess probability: 0.125
Probability of obsering 010 :  0.5003
Probability of obsering 101 :  0.4997


We observe that the Grover's Search outputs 101 and 010, which are the two colorings where there are 2 edges connecting vertices with different colors. 

In the remaining of this notebook, you will apply the same algorithm for other graphs. 

### Task 1

Implement the Grover algorithm for the following graph
<img src="../images/completebipartite.png" width="25%" align="center">

and check whether there is a coloring in which there are exactly four edges connecting vertices with a different color. Apply the oracle twice.

In [ ]:
import cirq
from cirq import H, Z, X, inverse, I, CX, CCX
s = cirq.Simulator()

# 0-3: vertices
# ?-?: edge checking
# ?-?: the number
# ?-?: auxillary

def oracle_computation():
    #
    # your solution goes here
    # 

def oracle():    
    #
    # your solution goes here
    # 
    
def grover_diffusion():
    #
    # your solution goes here
    # 

# the Grover algorithm
qq = cirq.LineQubit.range(12) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:4], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**4)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

[click for our solution](B04_Grover_Algorithm_For_Max-Cut_Problem_Solutions.ipynb#task1)

### Task 2

Implement the Grover Algorithm for the following graph
<img src="../images/finalgrover1.png" width="25%" align="center">

and check whether there is coloring with six or more edges connecting vertices with a different color. Apply the oracle twice. 

*Hint*: Note that you may check only two bits of the number.

In [ ]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

#
# your solution
# 

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**5)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

[click for our solution](B04_Grover_Algorithm_For_Max-Cut_Problem_Solutions.ipynb#task2)

## Final remarks

We have successfully implemented the Grover's Search algorithm for the Max-Cut problem. Congratulations! However, there are several interesting additional facts.

Currently for each edge, we have a separate qubit. We could reuse a single qubit at the cost of additional quantum gates. While the circuit would be longer, we could save some qubits.

Next, for each task, the number of iterations was given. While for bipartiteness checking we know there are always exactly two answers, this may not be the case for the Max-Cut problem. Fortunately, there is a *quantum counting algorithm*, which can efficiently determine the number of solutions and hence the number of iterations.